In [1]:
# --- Installation de MMDetection ---
# On supprime d'abord les versions pré-installées de torch et torchvision pour éviter les conflits
!pip uninstall -y torch torchvision
# On installe la version de PyTorch compatible avec les GPU de Kaggle
!pip install torch==2.0.0 torchvision==0.15.1 --extra-index-url https://download.pytorch.org/whl/cu118 -q

# Installation de MIM, le gestionnaire de paquets de MMLab
!pip install openmim -q

# --- CORRECTION : Forcer l'installation d'une version compatible de MMCV ---
# MMDetection (v3.x) requiert une version de MMCV < 2.2.0.
# Nous installons explicitement la version 2.1.0 qui est stable et compatible.
!mim install mmcv==2.1.0 -q

# Installer MMDetection
!mim install mmdet -q

# S'assurer que git est bien disponible
!apt-get install -y git

print("\n--- Versions des bibliothèques installées ---")
!python -c "import torch, torchvision; print('PyTorch:', torch.__version__); print('Torchvision:', torchvision.__version__)"
!python -c "import mmcv; print('MMCV:', mmcv.__version__)"
!python -c "import mmdet; print('MMDetection:', mmdet.__version__)"

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 456.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.2 requires torch>=2.1.0, but you have torch 2.0.0+cu118 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.0.0+cu118 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# --- Cloner le dépôt MMDetection ---
# On clone le dépôt pour avoir un accès fiable à tous les fichiers de configuration
!git clone https://github.com/open-mmlab/mmdetection.git
# On se déplace dans le dossier cloné pour faciliter les chemins relatifs
%cd mmdetection

print("Dépôt MMDetection cloné avec succès.")

Cloning into 'mmdetection'...
remote: Enumerating objects: 38023, done.
remote: Total 38023 (delta 0), reused 0 (delta 0), pack-reused 38023 (from 1)
Receiving objects: 100% (38023/38023), 63.27 MiB | 29.70 MiB/s, done.
Resolving deltas: 100% (26208/26208), done.
/kaggle/working/mmdetection
Dépôt MMDetection cloné avec succès.


In [3]:
import os
import json
import zipfile
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split # Utilisé pour créer un mini-val set si nécessaire

# --- Configuration ---
WORKING_DIR = Path('/kaggle/working/')
DATASET_DIR = Path('/kaggle/input/augmented-savi-640/Dataset_B_640x640')

# --- Fonction de Conversion YOLO -> COCO ---
def convert_yolo_to_coco(yolo_dataset_path, output_path):
    """
    Convertit un dataset au format YOLO en format COCO JSON.
    """
    output_path.mkdir(parents=True, exist_ok=True)
    
    for split in ["train", "val", "test"]:
        print(f"Conversion du split : {split}")
        split_images_path = yolo_dataset_path / "images" / split
        split_labels_path = yolo_dataset_path / "labels" / split
        
        coco_output = {
            "info": {
                "description": "Custom Dataset in COCO format",
                "version": "1.0",
                "year": 2025
            },
            "licenses": [],
            "images": [],
            "annotations": [],
            "categories": [
                {"id": 0, "name": "Person"},
                {"id": 1, "name": "Bicycle"},
                {"id": 2, "name": "Car"},
                {"id": 3, "name": "Cattle"},
            ]
        }
        
        image_id_counter = 0
        annotation_id_counter = 0
        
        image_files = sorted(list(split_images_path.glob("*.jpg")))
        
        for img_path in tqdm(image_files, desc=f"Processing {split} images"):
            with Image.open(img_path) as img:
                img_w, img_h = img.size
            
            # Ajouter l'info de l'image
            coco_output["images"].append({
                "id": image_id_counter,
                "file_name": str(img_path.name),
                "width": img_w,
                "height": img_h
            })
            
            # Ajouter les annotations
            label_path = split_labels_path / (img_path.stem + ".txt")
            if label_path.exists():
                with open(label_path, 'r') as f:
                    for line in f:
                        class_id, x_c, y_c, w, h = map(float, line.strip().split())
                        
                        # Conversion de YOLO [x_center, y_center, w, h]
                        # en COCO [x_min, y_min, w, h]
                        box_w = w * img_w
                        box_h = h * img_h
                        x_min = (x_c * img_w) - (box_w / 2)
                        y_min = (y_c * img_h) - (box_h / 2)
                        
                        coco_output["annotations"].append({
                            "id": annotation_id_counter,
                            "image_id": image_id_counter,
                            "category_id": int(class_id),
                            "bbox": [x_min, y_min, box_w, box_h],
                            "area": box_w * box_h,
                            "iscrowd": 0
                        })
                        annotation_id_counter += 1
            
            image_id_counter += 1
            
        # Sauvegarder le fichier JSON pour ce split
        json_path = output_path / f"{split}.json"
        with open(json_path, 'w') as f:
            json.dump(coco_output, f, indent=4)
        print(f"Fichier {json_path} créé avec succès.")

# --- Lancement de la Conversion ---
COCO_ANNOTATIONS_DIR = WORKING_DIR / "coco_annotations"
convert_yolo_to_coco(DATASET_DIR, COCO_ANNOTATIONS_DIR)

Conversion du split : train


Processing train images: 100%|██████████| 2660/2660 [00:28<00:00, 91.90it/s]


Fichier /kaggle/working/coco_annotations/train.json créé avec succès.
Conversion du split : val


Processing val images: 100%|██████████| 296/296 [00:03<00:00, 91.59it/s]


Fichier /kaggle/working/coco_annotations/val.json créé avec succès.
Conversion du split : test


Processing test images: 100%|██████████| 888/888 [00:09<00:00, 92.78it/s]

Fichier /kaggle/working/coco_annotations/test.json créé avec succès.


In [4]:
from mmengine import Config
import json
from pathlib import Path

# --- Création du Fichier de Configuration Personnalisé ---

# Définir le contenu de notre fichier de configuration.
# Cette approche est beaucoup plus robuste que de modifier l'objet cfg en mémoire.

# Chemin vers notre dossier de données principal
data_root_for_images = str(DATASET_DIR) + '/'
annotations_root_for_json = str(COCO_ANNOTATIONS_DIR) + '/'

# Noms de nos classes
class_names = ('Person', 'Bicycle', 'Car', 'Cattle')

# Contenu du fichier de config
custom_config_content = f"""
# --- Héritage de la configuration de base ---
_base_ = [
    '/kaggle/working/mmdetection/configs/cascade_rcnn/cascade-rcnn_r50_fpn_1x_coco.py'
]

# --- 1. Modification des Paramètres du Dataset ---
dataset_type = 'CocoDataset'
data_root = '{data_root_for_images}'

# Définir les métadonnées (noms de classes) pour le dataset
metainfo = dict(classes={class_names})

# Redéfinir complètement les dataloaders
train_dataloader = dict(
    batch_size=2,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=metainfo,
        ann_file='{annotations_root_for_json}train.json',
        data_prefix=dict(img='images/train/')))

val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=metainfo,
        ann_file='{annotations_root_for_json}val.json',
        data_prefix=dict(img='images/val/')))

test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=metainfo,
        ann_file='{annotations_root_for_json}test.json',
        data_prefix=dict(img='images/test/')))

# Redéfinir les évaluateurs de validation et de test
val_evaluator = dict(ann_file='{annotations_root_for_json}val.json')
test_evaluator = dict(ann_file='{annotations_root_for_json}test.json')


# --- 2. Modification des Paramètres du Modèle ---
model = dict(
    roi_head=dict(
        bbox_head=[
            dict(type='Shared2FCBBoxHead', num_classes=4),
            dict(type='Shared2FCBBoxHead', num_classes=4),
            dict(type='Shared2FCBBoxHead', num_classes=4)
        ]))

# --- 3. Modification des Paramètres d'Exécution ---
# Dossier de travail pour sauvegarder les logs et les poids
work_dir = '/kaggle/working/cascade_rcnn_baseline'

# Charger des poids pré-entraînés sur COCO
load_from = 'https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth'

# Définir la boucle d'entraînement
train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=36, val_interval=1)

# Configurer les checkpoints pour sauvegarder le meilleur modèle
default_hooks = dict(
    checkpoint=dict(type='CheckpointHook', interval=1, save_best='auto', rule='greater')
)
"""

# Écrire ce contenu dans un fichier Python
custom_config_path = WORKING_DIR / 'cascade_rcnn_custom_config.py'
with open(custom_config_path, 'w') as f:
    f.write(custom_config_content)

print(f"Fichier de configuration personnalisé créé avec succès : {custom_config_path}")

# --- Étape de Débogage cruciale : Vérifier les chemins ---
# Charger la configuration que nous venons de créer
cfg = Config.fromfile(custom_config_path)

# Vérifier le chemin du fichier d'annotation
print("\\n--- VÉRIFICATION DES CHEMINS D'ANNOTATION ---")
ann_file_path = Path(cfg.train_dataloader.dataset.ann_file)
print(f"Chemin de l'annotation tel que configuré : {ann_file_path}")
print(f"Ce chemin existe-t-il ? -> {ann_file_path.exists()}")

# Vérifier le chemin d'une image
print("\\n--- VÉRIFICATION DES CHEMINS D'IMAGE ---")
train_json_path = ann_file_path # Utiliser le chemin déjà vérifié
with open(train_json_path, 'r') as f:
    train_data = json.load(f)

if train_data['images']:
    first_image_filename = train_data['images'][0]['file_name']
    
    # Reconstruire le chemin complet tel que MMDetection le verra
    full_image_path = Path(cfg.data_root) / cfg.train_dataloader.dataset.data_prefix['img'] / first_image_filename
    
    print("\n--- VÉRIFICATION DES CHEMINS ---")
    print(f"Chemin de l'image tel que reconstruit : {full_image_path}")
    print(f"Ce chemin existe-t-il ? -> {full_image_path.exists()}")
    if not full_image_path.exists():
        print("ERREUR DE DÉBOGAGE : Le chemin vers les images est incorrect. Vérifiez les variables 'data_root' et 'data_prefix'.")
else:
    print("AVERTISSEMENT : Le fichier train.json ne contient aucune image.")

Fichier de configuration personnalisé créé avec succès : /kaggle/working/cascade_rcnn_custom_config.py
\n--- VÉRIFICATION DES CHEMINS D'ANNOTATION ---
Chemin de l'annotation tel que configuré : /kaggle/working/coco_annotations/train.json
Ce chemin existe-t-il ? -> True
\n--- VÉRIFICATION DES CHEMINS D'IMAGE ---

--- VÉRIFICATION DES CHEMINS ---
Chemin de l'image tel que reconstruit : /kaggle/input/augmented-savi-640/Dataset_B_640x640/images/train/HIT-UAV_0_100_30_0_03289.jpg
Ce chemin existe-t-il ? -> True


In [5]:
from mmengine.runner import Runner

# --- Lancement de l'Entraînement ---

# L'objet 'cfg' a déjà été chargé et vérifié dans la cellule précédente
runner = Runner.from_cfg(cfg)

# Démarrer l'entraînement
print("Début de l'entraînement de Cascade R-CNN...")
runner.train()
print("Entraînement terminé.")

Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0+cu118


09/24 21:58:48 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 930151935
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2025.2-Product Build 20250620 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


09/24 21:58:52 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth


Downloading: "https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth" to /root/.cache/torch/hub/checkpoints/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth


The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.0.fc_reg.weight: copying a param with shape torch.Size([4, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_head.bbox_head.0.fc_reg.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([16]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param with shape torch.Size([81]) from che

In [6]:
import glob

# --- Évaluation sur le Test Set ---

# Trouver le chemin du meilleur checkpoint sauvegardé
# Il se termine par 'best_coco_bbox_mAP...pth'
work_dir = cfg.work_dir
best_checkpoint_list = glob.glob(str(Path(work_dir) / 'best_*.pth'))

if best_checkpoint_list:
    best_checkpoint = best_checkpoint_list[0]
    print(f"Meilleur checkpoint trouvé : {best_checkpoint}")
    
    # Mettre à jour la config pour charger ce checkpoint spécifique
    cfg.load_from = best_checkpoint
    
    # Créer un nouveau runner pour le test
    runner = Runner.from_cfg(cfg)
    
    # Lancer l'évaluation
    print("\nLancement de l'évaluation sur l'ensemble de test...")
    test_metrics = runner.test()
    
    print("\n--- Métriques sur le Test Set ---")
    print(test_metrics)
else:
    print("Aucun 'meilleur' checkpoint n'a été trouvé. Vérifiez les logs d'entraînement.")

Meilleur checkpoint trouvé : /kaggle/working/cascade_rcnn_baseline/best_coco_bbox_mAP_epoch_29.pth
09/25 02:21:19 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 2017822294
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2025.2-Product Build 20250620 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - 

/usr/local/lib/python3.11/dist-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmdet.visualization.local_visualizer.DetLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


09/25 02:21:20 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
09/25 02:21:20 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

In [7]:
from mmdet.apis import init_detector, inference_detector
import mmcv
import matplotlib.pyplot as plt

# --- Inférence et Visualisation ---

# Initialiser le modèle avec la config et le meilleur checkpoint
model = init_detector(str(custom_config_path), best_checkpoint, device='cuda:0')

# Sélectionner quelques images de test aléatoires
test_images_dir = DATASET_DIR / 'images' / 'test'
test_image_files = list(test_images_dir.glob('*.jpg'))
random_test_images = random.sample(test_image_files, k=12)

print(f"Affichage des prédictions sur {len(random_test_images)} images de test aléatoires...")

for img_path in random_test_images:
    # Effectuer l'inférence
    result = inference_detector(model, str(img_path))
    
    # Visualiser les résultats
    # MMDetection peut dessiner les boîtes directement sur l'image
    img_with_boxes = model.show_result(
        str(img_path),
        result,
        score_thr=0.5 # N'afficher que les détections avec un score > 0.5
    )
    
    # Afficher dans le notebook
    plt.figure(figsize=(10, 10))
    # mmcv charge les images en BGR, il faut convertir en RGB pour matplotlib
    plt.imshow(mmcv.bgr2rgb(img_with_boxes))
    plt.title(img_path.name)
    plt.axis('off')
    plt.show()

Loads checkpoint by local backend from path: /kaggle/working/cascade_rcnn_baseline/best_coco_bbox_mAP_epoch_29.pth


NameError: name 'random' is not defined